# What happens:

Entity_2_yyyyymmdd.xlsx file is created where all you need to do is upload it to RIC then

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import win32com.client as win32
from openpyxl import load_workbook

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0194"

### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Save selection
### Also see how much time it took

In [4]:
%%time
d = pd.read_sql_query("""WITH OUTCOME as ( select r.booking_company, r.counterparty_original as counterparty, e1.entity_desc, 
g1.entity_Code as gcc_code, g1.entity_desc as gcc_name, g1.total_annual_sales as gcc_tas, r.asset_class_original as asset_Class,
r.bis_entity_type_original as entity_type, e1.et_code_user, et1.entity_type_desc, i.long_term_Rating, sum(ead_pre_ccf) exposure,
sum(rwa) rwa from t_cdr r left join entity e1 on r.counterparty_original = e1.entity_code 
left join entity_type et1 on e1.et_code_user = et1.entity_type 
left join issuer_Credit_ratings i on r.counterparty_original = i.entity_code 
left join esr s on e1.entity_code = s.entity_code 
left join entity g1 on s.entity_group = g1.entity_code 
where counterparty_original in ( select s.entity_code from esr s where s.original_entity_class <> s.actual_entity_class and s.actual_entity_class in ('SME','SME_PF') ) and rwa > 0 and i.long_term_rating not like 'CORP%' and i.long_term_rating not like 'SME%' and i.long_term_rating not like 'SLOT%' group by g1.entity_code, g1.entity_desc, g1.total_annual_Sales, r.counterparty_original, r.asset_class_original, r.bis_entity_type_original, e1.et_code_user, et1.entity_type_desc, e1.entity_desc, r.booking_company, i.long_term_Rating UNION select r.booking_company, r.counterparty_new as counterparty, e1.entity_desc, g2.entity_code as gcc_code, g2.entity_desc as gcc_name, g2.total_annual_sales as gcc_tas, r.asset_class_new as asset_class, r.bis_entity_type_new as entity_type, e1.et_code_user, et1.entity_type_desc, i.long_term_Rating, sum(ead_pre_ccf) exposure, sum(rwa) rwa from t_cdr r left join entity e1 on r.counterparty_new = e1.entity_code left join entity_type et1 on e1.et_code_user = et1.entity_type left join issuer_Credit_ratings i on r.counterparty_new = i.entity_code left join esr s on e1.entity_code = s.entity_code left join entity g2 on s.entity_group = g2.entity_code where counterparty_new in ( select s.entity_code from esr s where s.original_entity_class <> s.actual_entity_class and s.actual_entity_class in ('SME','SME_PF') ) and rwa > 0 and i.long_term_rating not like 'CORP%' and i.long_term_rating not like 'SME%' and i.long_term_rating not like 'SLOT%' group by g2.entity_code, g2.entity_desc, g2.total_annual_Sales, r.counterparty_new, r.asset_class_new, r.bis_entity_type_new, e1.et_code_user, et1.entity_type_desc, e1.entity_desc, r.booking_company, i.long_term_Rating ) select o.booking_company, o.counterparty, o.entity_desc as counterparty_name, o.et_code_user, o.entity_type_desc, o.long_term_rating, o.gcc_code, o.gcc_name, o.gcc_tas, o.exposure, o.rwa, case when (s.actual_entity_class = 'SME' and o.entity_type = 'SME_EU') then 'Firm Size Adjustement and SME support factor applied' when (s.actual_entity_class = 'SME' and o.entity_type <> 'SME_EU') then 'Firm Size Adjustement applied' else '**Error**' end as treatment from OUTCOME o left join esr s on o.counterparty = s.entity_Code order by treatment
""", connection_RAY)

CPU times: total: 0 ns
Wall time: 16 s


### Save var in an excel file

In [9]:
# determining the name of the file
file_name = rd + '_2nd_Tas_MC.xlsx'
ex_folder = "Export/"
im_folder = "Import/"

# saving the excel
d.to_excel(ex_folder + file_name, index=False)
print('DataFrame is written to Excel File successfully.')

#load saved excel file
file_saved = load_workbook( filename = ex_folder + file_name)
#open workbook
file_saved_sheet = file_saved.active


#load saved entity file
entity = load_workbook( filename = im_folder + "ENTITY_2_TEMPLATE.xlsm")
#open workbook
entity_sheet = entity.active



DataFrame is written to Excel File successfully.


C:\Users\WZHARBC\AppData\Roaming\Python\Python310\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [10]:
rng = range( 2, file_saved_sheet.max_row+1, 1)
gcc_codes = []
for n in rng:
    if file_saved_sheet.cell(row = n, column = 7).value not in gcc_codes:
        gcc_codes.append(file_saved_sheet.cell(row = n, column = 7).value)

rng_2 = range( 19, len(gcc_codes) + 19, 1)
for n in rng_2:
    entity_sheet.cell( row = n, column = 1).value = gcc_codes[n-19]
    entity_sheet.cell( row = n, column = 7).value = -2
    
#deleting extra file
os.remove("Export/" + rd + "_2nd_Tas_MC.xlsx")

#save the file
entity.save(filename="Export/Entity_2_" + rd + ".xlsx")
print("File saved")
print("Upload file to RIC")

File saved
Upload file to RIC
